<a href="https://colab.research.google.com/github/osgeokr/kari-sdm/blob/main/240726_npp_analysis_cotubanama_national_park.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

ee.Authenticate()

ee.Initialize(project='ee-foss4g')

In [50]:
# Cotubanama National Park
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
# wdpa = wdpa.filter(ee.Filter.eq('WDPAID', 180))
wdpa = wdpa.filter(ee.Filter.eq('WDPAID', 768))

wdpa_name = wdpa.first().get('NAME').getInfo()
print("Name:", wdpa_name)

Name: Seoraksan


In [51]:
# 1: Pixel value for the outline, 2: Thickness of the outline (in pixels)
wdpa_raster = ee.Image().paint(wdpa, 1, 2)

# Set visualization parameters
wdpa_vis = {
    'palette': ['#FFFF00'], # Apply Yellow color
    'opacity': 1
}

m = geemap.Map(width="800px", height="400px")
m.addLayer(wdpa_raster, wdpa_vis, wdpa_name)
m.centerObject(wdpa, 10)
m

Map(center=[38.13576002082187, 128.41324042116062], controls=(WidgetControl(options=['position', 'transparent_…

In [52]:
dataset = (
    ee.ImageCollection('MODIS/061/MOD17A3HGF')
    .filterDate("2023-01-01", "2023-12-31")
    .filterBounds(wdpa)
)

npp = dataset.select('Npp').first().clip(wdpa.geometry()).multiply(0.0001)

crs = ee.Projection('EPSG:5179')
npp = npp.reproject(crs=crs, scale=500)

npp_vis = {
    'min': 0,
    'max': 1.9,
    'palette': ['bbe029', '0a9501', '074b03']
}

m = geemap.Map(width="800px", height="400px",
               basemap='Esri.WorldImagery')
m.addLayer(npp, npp_vis, 'NPP')
m.addLayer(wdpa_raster, wdpa_vis, wdpa_name)
m.centerObject(wdpa, 10)
m

Map(center=[38.13576002082187, 128.41324042116062], controls=(WidgetControl(options=['position', 'transparent_…

In [54]:
# NPP 픽셀 값 합계 계산
npp_sum = npp.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=wdpa.geometry(),
    scale=500,
    maxPixels=1e13
).get('Npp').getInfo()

total_npp = npp_sum * 250000 * 0.001

# 결과 출력
print(f"{total_npp:.2f} Mg")

318388.60 Mg
